# Estrutura:

* [Tratamento de Dados](#dados)
* [Integração com o Antigo Notebook](#integracao)

# Data <a id="dados"><a/>

In [106]:
# https://www.analyticsvidhya.com/blog/2021/06/image-recognition-using-pytorch-lightning/
# https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
import torchvision
import cv2
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
import os
from random import randint
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from datetime import datetime
from pathlib import Path
import tempfile
from glob import glob

import torch
import monai
import pandas as pd
import torchio as tio
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
from config import *
import albumentations as A
monai.config.print_config()

# Load dataset <a id = "torchdataset"><a/>

In [128]:
# Data train 
path_dir_train = 'datasets/diaretdb1_v_1_1/TRAINSET/ddb1_fundusimages'
path_mask_train = 'datasets/diaretdb1_v_1_1/TRAINSET/ddb1_groundtruth/hardexudates'
img_list_train = os.listdir(os.path.join(path_dir_train))

# Data test
path_dir_test = 'datasets/diaretdb1_v_1_1/TESTSET/ddb1_fundusimages'
path_mask_test = 'datasets/diaretdb1_v_1_1/TESTSET/ddb1_groundtruth/hardexudates'
img_list_test = os.listdir(os.path.join(path_dir_test))

In [129]:
transformations = {
    'test': transforms.Compose([
                        transforms.ToPILImage(),
                        transforms.Resize((224,224)),
                        # transforms.Grayscale(1),
                        transforms.ToTensor()]),
     'train': transforms.Compose([
                           transforms.ToPILImage(),
                           transforms.Resize((224,224)),
                           transforms.RandomRotation(degrees = 45),
                           transforms.RandomHorizontalFlip(p = 0.005),
                        #    transforms.Grayscale(1),
                           transforms.ToTensor(),
                           A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                           ])
}

In [110]:

class RD_SegmentationDataset(Dataset):
    def __init__(self, img_list, image_dir, mask_dir, transform=None, target_transform=None):
        
        self.data_list = img_list
        self.data_img = image_dir
        self.data_mask = mask_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):

        path_image = os.path.join(self.data_img, self.data_list[idx])
        image = cv2.imread(path_image)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    
        h, w, c = np.shape(image)
        image= image.reshape(c,h,w)
        print(image.shape)
        path_mask = os.path.join(self.data_mask, self.data_list[idx])
        mask = cv2.imread(path_mask,0) 
        # h1, w1 = np.shape(mask)
        # mask = np.reshape(h1,w1)
        mask = cv2.threshold(mask, 240, 255, cv2.THRESH_BINARY)[1]
        mask = mask/255
        mask = np.expand_dims(mask, 2)


        
        # if self.transform:   
        #     # transformed = self.transform(image=image, mask=mask) 
        #     transformed = self.transform(image=image, mask=mask) 
        #     image = transformed["image"] 
        #     mask = transformed["mask"] 
        #     mask = mask.permute(2,0,1)


        return {'image':image, 'mask':mask}
    
RD_dataset = RD_SegmentationDataset(img_list_train, path_dir_train, path_mask_train, transformations['train'])
RD_dataloader = DataLoader(RD_dataset, batch_size=4, shuffle=True)

In [123]:
# Get a batch of training data
batch = next(iter(RD_dataloader))
inputs = batch['image']
print(inputs.dtype)
masks = batch['mask']

# fig, axs  = plt.subplots(4,2, figsize=(8,10))
# for ax, i in zip(axs, range(len(axs))):
#     ax[0].imshow(inputs[i])
#     ax[1].imshow(masks[i], cmap="gray")
# fig.suptitle("Lote de amostra de treino")
# plt.show()


(3, 576, 750)
(3, 576, 750)
(3, 576, 750)
(3, 576, 750)
torch.uint8


# Setup Enviroment <a id="integracao"><a/>

In [186]:
!python -c "import monai" || pip install -q "monai-weekly"
!pip install -q torch==1.10.2 torchtext==0.11.2 torchvision==0.11.3
!pip install -q torchio==0.18.73
!pip install -q pytorch-lightning==1.5.10
!pip install -q pandas==1.1.5 seaborn==0.11.1
!pip install -q pillow==9.4.0

^C


# Configurations and Tensorboard Setup

In [113]:
sns.set()
plt.rcParams["figure.figsize"] = 12, 8
monai.utils.set_determinism()

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Setup Data Directory

In [112]:
# directory = os.environ.get("MONAI_DATA_DIRECTORY")
# dataset_dir = tempfile.mkdtemp() if directory is None else directory
# print(dataset_dir)

dataset_dir = "./datasets/diaretdb1_v_1_1"
print(dataset_dir)

./datasets/diaretdb1_v_1_1


# Data

In [9]:
# class MedicalDecathlonDataModule(pl.LightningDataModule):
#     def __init__(self, dataset_name, batch_size, train_val_ratio, lesion):
#         '''lesion: EX: hardexsudates, HE: hemorrhages, MA: microaneurysms, SE'''
#         super().__init__()
#         self.dataset_name = dataset_name
#         self.batch_size = batch_size
#         self.base_dir = root_dir
#         self.dataset_dir = os.path.join(root_dir, dataset_name)
#         self.lesion = lesion
#         self.train_val_ratio = train_val_ratio
#         self.subjects = None
#         self.test_subjects = None
#         self.preprocess = None
#         self.transform = None
#         self.train_set = None
#         self.val_set = None
#         self.test_set = None

    # def download_data(self):
    #     if not os.path.isdir(self.dataset_dir):
    #         url = "https://www.it.lut.fi/project/imageret/diaretdb1/diaretdb1_v_1_1.zip"
    #         monai.apps.download_and_extract(url, output_dir="./datasets")
    #         # TESTSET:
    #         adaptar_dataset(ROOT_DATASET_PATH, IMGS_FUNDUS_PATH, MASKS_DIR_PATH, ANNOTATIONS_TEST_PATH)
    #         # TRAINSET:
    #         adaptar_dataset(ROOT_DATASET_PATH, IMGS_FUNDUS_PATH, MASKS_DIR_PATH, ANNOTATIONS_TRAIN_PATH)

    #     mask_dir = sorted(os.listdir(os.path.join(self.dataset_dir, "TRAINSET", "ddb1_groundtruth")))[LESIONS[self.lesion]]

    #     image_training_paths = sorted(glob(os.path.join(self.dataset_dir, "TRAINSET", "ddb1_fundusimages", "*.png")))
    #     label_training_paths = sorted(glob(os.path.join(self.dataset_dir, "TRAINSET", "ddb1_groundtruth", mask_dir, "*.png")))
    #     image_test_paths = sorted(glob(os.path.join(self.dataset_dir, "TESTSET", "ddb1_fundusimages", "*.png*")))
    #     print(image_training_paths, label_training_paths, image_test_paths)
    #     return image_training_paths, label_training_paths, image_test_paths
    
    
    # def prepare_data(self):
    #     '''como baixar, tokenizar, etc…'''
    #     image_training_paths, label_training_paths, image_test_paths = self.download_data()

    #     self.subjects = []
    #     for image_path, label_path in zip(image_training_paths, label_training_paths):
    #         # 'image' and 'label' are arbitrary names for the images
    #         subject = tio.Subject(image=tio.ScalarImage(image_path), label=tio.LabelMap(label_path))
    #         self.subjects.append(subject)

    #     self.test_subjects = []
    #     for image_path in image_test_paths:
    #         subject = tio.Subject(image=tio.ScalarImage(image_path))
    #         self.test_subjects.append(subject)

    # def get_preprocessing_transform(self):
    #     preprocess = tio.Compose(
    #         [
    #             tio.RescaleIntensity((0, 1)),
    #             # tio.CropOrPad(self.get_max_shape(self.subjects + self.test_subjects)),
    #             tio.EnsureShapeMultiple(8),  # for the U-Net
    #             tio.OneHot(),
    #         ]
    #     )
    #     return preprocess

    # def get_augmentation_transform(self):
    #     augment = tio.Compose(
    #         [
    #             tio.RandomAffine(),
    #             # tio.RandomGamma(p=0.5),
    #             # tio.RandomNoise(p=0.5),
    #             # tio.RandomMotion(p=0.1),
    #             # tio.RandomBiasField(p=0.25),
    #         ]
    #     )
    #     return augment
 

    # def setup(self, stage=None):
    #     '''como dividir, definir conjunto de dados, etc…'''
    #     # self.transforms =   data_transforms = { 
    #     #                     'val': A.Compose([ 
    #     #                         A.Resize(224, 224),  
    #     #                         A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  
    #     #                         # ToTensorV2() 
    #     #                     ]), 
    #     #                     }
    #     num_subjects = len(self.subjects)
    #     num_train_subjects = int(round(num_subjects * self.train_val_ratio))
    #     num_val_subjects = num_subjects - num_train_subjects
    #     splits = num_train_subjects, num_val_subjects
    #     train_subjects, val_subjects = random_split(self.subjects, splits)

    #     self.preprocess = self.get_preprocessing_transform()
    #     augment = self.get_augmentation_transform()
    #     self.transform = tio.Compose([self.preprocess, augment])

    #     self.train_set = tio.SubjectsDataset(train_subjects, transform=self.transform)
    #     self.val_set = tio.SubjectsDataset(val_subjects, transform=self.preprocess)
    #     self.test_set = tio.SubjectsDataset(self.test_subjects, transform=self.preprocess)

    # def train_dataloader(self):
    #     return DataLoader(self.train_set, self.batch_size, num_workers=2)

    # def val_dataloader(self):
    #     return DataLoader(self.val_set, self.batch_size, num_workers=2)

    # def test_dataloader(self):
    #     return DataLoader(self.test_set, self.batch_size, num_workers=2)

[Definição da Classe do Dataset](#torchdataset)

In [126]:
train_set = RD_SegmentationDataset(img_list = img_list_train, 
                                   image_dir = path_dir_train, 
                                   mask_dir = path_mask_train, 
                                   transform = transformations['train'])
test_set = RD_SegmentationDataset(img_list = img_list_test, 
                                   image_dir = path_dir_test, 
                                   mask_dir = path_mask_test, 
                                   transform = transformations['test'])

train_loader = DataLoader(train_set, batch_size=4, shuffle=True)
test_loader = DataLoader(test_set, batch_size=4, shuffle=True)

# Lightning model

In [127]:
class Model(pl.LightningModule):
    def __init__(self, net, criterion, learning_rate, optimizer_class):
        super().__init__()
        self.lr = learning_rate
        self.net = net
        self.criterion = criterion
        self.optimizer_class = optimizer_class

    def configure_optimizers(self):
        optimizer = self.optimizer_class(self.parameters(), lr=self.lr)
        return optimizer

    def prepare_batch(self, batch):

        return batch["image"] , batch["mask"]

    def infer_batch(self, batch):
        x, y = self.prepare_batch(batch)
        print(x.dtype, y.dtype)
        y_hat = self.net(x)
        return y_hat, y

    def training_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        y_hat, y = self.infer_batch(batch)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

In [124]:
unet = monai.networks.nets.UNet(
    spatial_dims=2,
    in_channels=3,
    out_channels=2,
    channels=(8, 16, 32, 64),
    strides=(2, 2, 2),
)

model = Model(
    net=unet,
    criterion=monai.losses.DiceCELoss(softmax=True),
    learning_rate=1e-2,
    optimizer_class=torch.optim.AdamW,
)
early_stopping = pl.callbacks.early_stopping.EarlyStopping(
    monitor="val_loss",
)
trainer = pl.Trainer(
    gpus=0,
    # precision='bf16',
    callbacks=[early_stopping]
)
trainer.logger._default_hp_metric = False

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


# Training

In [125]:
start = datetime.now()
print("Training started at", start)
trainer.fit(model=model, train_dataloaders=train_loader)

print("Training duration:", datetime.now() - start)

c:\Users\LESC\Desktop\UNet-SegRD\lesc-env\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name      | Type       | Params
-----------------------------------------
0 | net       | UNet       | 41.0 K
1 | criterion | DiceCELoss | 0     
-----------------------------------------
41.0 K    Trainable params
0         Non-trainable params
41.0 K    Total params
0.164     Total estimated model params size (MB)


Training started at 2023-04-05 16:11:50.982345


c:\Users\LESC\Desktop\UNet-SegRD\lesc-env\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\LESC\Desktop\UNet-SegRD\lesc-env\lib\site-packages\pytorch_lightning\trainer\data_loading.py:432: UserWarning: The number of training samples (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

(3, 576, 750)
(3, 576, 750)
(3, 576, 750)
(3, 576, 750)
(3, 576, 750)
(3, 576, 750)
(3, 576, 750)
(3, 576, 750)
<class 'torch.Tensor'> <class 'torch.Tensor'>


RuntimeError: expected scalar type Byte but found Float

In [ ]:
%tensorboard --logdir lightning_logs

# Plot validation results

In [ ]:
model.to("cpu")
all_dices = []
get_dice = monai.metrics.DiceMetric(include_background=False, reduction="none")
with torch.no_grad():
    for batch in data.val_dataloader():
        inputs, targets = model.prepare_batch(batch)
        logits = model.net(inputs.to(model.device))
        labels = logits.argmax(dim=1)
        labels_one_hot = torch.nn.functional.one_hot(labels).permute(0, 4, 1, 2, 3)
        get_dice(labels_one_hot.to(model.device), targets.to(model.device))
    metric = get_dice.aggregate()
    get_dice.reset()
    all_dices.append(metric)
all_dices = torch.cat(all_dices)

In [ ]:
records = []
for ant, post in all_dices:
    records.append({"Dice": ant, "Label": "Anterior"})
    records.append({"Dice": post, "Label": "Posterior"})
df = pd.DataFrame.from_records(records)
ax = sns.stripplot(x="Label", y="Dice", data=df, size=10, alpha=0.5)
ax.set_title("Dice scores")

# Test

In [ ]:
with torch.no_grad():
    for batch in data.test_dataloader():
        inputs = batch["image"][tio.DATA].to(model.device)
        labels = model.net(inputs).argmax(dim=1, keepdim=True).cpu()
        break
batch_subjects = tio.utils.get_subjects_from_batch(batch)
tio.utils.add_images_from_batch(batch_subjects, labels, tio.LabelMap)

In [ ]:
for subject in batch_subjects:
    subject.plot()